In [1]:
import torch

In [2]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU device name:", torch.cuda.get_device_name(0))

PyTorch version: 2.8.0+cu126
CUDA available: True
CUDA device count: 1
GPU device name: NVIDIA GeForce RTX 4060 Laptop GPU
